In [1]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

In [2]:
# 由 Keras 讀入 MNIST
from keras.datasets import mnist

Using TensorFlow backend.


In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
# 輸入格式整理
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)

# 正規化 將資料線性單位化至  [0,1] 。
x_train = (x_train - x_train.min()) / (x_train.max() - x_train.min())
x_test = (x_test - x_test.min()) / (x_test.max() - x_test.min())

In [5]:
x_train.min(), x_train.max()

(0.0, 1.0)

In [6]:
# 輸出格式整理 
from keras.utils import np_utils

In [7]:
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

In [8]:
# 決定神經網路架構、讀入相關套件
from keras.layers import Dense, Activation
from keras.optimizers import SGD

# Functional API

In [9]:
from keras.models import Model
from keras.layers import Input

In [10]:
# 建構我們的神經網路
f1 = Dense(120, activation='relu')
f2 = Dense(20, activation='relu')
f3 = Dense(10, activation='softmax')

In [11]:
# 第一個變數必定以 Input 函數來定義
x = Input(shape=(784,))

$$h_1 = f_1(x), h_2 = f_2(h_1), y = f_3(h_2)$$

In [12]:
# 直接照著數學式寫程式
h1 = f1(x)
h2 = f2(h1)
y = f3(h2)

Instructions for updating:
Colocations handled automatically by placer.


In [13]:
# 透過 Model 將一個模型的輸入/輸出包裝起來
model = Model(x, y)

In [14]:
# 組裝
model.compile(loss='mse', optimizer = SGD(lr = 0.3), metrics = ['accuracy'])
# 檢視我們的神經網路
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 120)               94200     
_________________________________________________________________
dense_2 (Dense)              (None, 20)                2420      
_________________________________________________________________
dense_3 (Dense)              (None, 10)                210       
Total params: 96,830
Trainable params: 96,830
Non-trainable params: 0
_________________________________________________________________


In [15]:
# 訓練神經網路
model.fit(x_train, y_train, batch_size = 100, epochs = 20, verbose = 1, validation_data = (x_test, y_test))

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 3s 54us/step - loss: 0.0598 - acc: 0.5493 - val_loss: 0.0254 - val_acc: 0.8534
Epoch 2/20
60000/60000 [==============================] - 3s 45us/step - loss: 0.0200 - acc: 0.8787 - val_loss: 0.0157 - val_acc: 0.9036
Epoch 3/20
60000/60000 [==============================] - 3s 50us/step - loss: 0.0151 - acc: 0.9037 - val_loss: 0.0132 - val_acc: 0.9193
Epoch 4/20
60000/60000 [==============================] - 3s 51us/step - loss: 0.0132 - acc: 0.9146 - val_loss: 0.0118 - val_acc: 0.9256
Epoch 5/20
60000/60000 [==============================] - 3s 49us/step - loss: 0.0120 - acc: 0.9231 - val_loss: 0.0111 - val_acc: 0.9311
Epoch 6/20
60000/60000 [==============================] - 3s 44us/step - loss: 0.0111 - acc: 0.9289 - val_loss: 0.0107 - val_acc: 0.9326
Epoch 7/20
60000/60000 [==============================] - 3s 44us/step - loss: 0

In [16]:
# 檢視準確率
score = model.evaluate(x_train, y_train, batch_size=10000)
print("Loss: %f" %score[0])
print("準確率: %f" %(score[1]*100))

60000/60000 [==============================] - 1s 16us/step
Loss: 0.005309
準確率: 96.850000


# 非線性堆疊模型
# 建立具分歧及合併結構的神經網路模型

In [17]:
from keras.layers import concatenate, add

In [18]:
# 定義  𝑓3 ,  𝑓4  及  𝑧
f4 = Dense(500, activation='relu')
z = f4(h1)

# new f3
f3 = Dense(10, activation='softmax')

In [19]:
# 將  ℎ2  與  𝑧  concatenate 接在一起，稱做  𝑢 。
u = concatenate([h2, z])
y = f3(u)

In [20]:
# 透過 Model 將一個模型的輸入/輸出包裝起來
model = Model(x, y)

In [21]:
# 組裝
model.compile(loss='mse', optimizer = SGD(lr = 0.3), metrics = ['accuracy'])
# 檢視我們的神經網路
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 784)          0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 120)          94200       input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 20)           2420        dense_1[0][0]                    
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 500)          60500       dense_1[0][0]                    
__________________________________________________________________________________________________
concatenat

In [22]:
# 訓練神經網路
model.fit(x_train, y_train, batch_size = 100, epochs = 20, verbose = 1, validation_data = (x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 5s 77us/step - loss: 0.0199 - acc: 0.8885 - val_loss: 0.0090 - val_acc: 0.9463
Epoch 2/20
60000/60000 [==============================] - 4s 71us/step - loss: 0.0079 - acc: 0.9550 - val_loss: 0.0075 - val_acc: 0.9530
Epoch 3/20
60000/60000 [==============================] - 4s 70us/step - loss: 0.0069 - acc: 0.9598 - val_loss: 0.0070 - val_acc: 0.9567
Epoch 4/20
60000/60000 [==============================] - 4s 72us/step - loss: 0.0063 - acc: 0.9631 - val_loss: 0.0066 - val_acc: 0.9601
Epoch 5/20
60000/60000 [==============================] - 4s 69us/step - loss: 0.0059 - acc: 0.9648 - val_loss: 0.0064 - val_acc: 0.9608
Epoch 6/20
60000/60000 [==============================] - 4s 72us/step - loss: 0.0056 - acc: 0.9667 - val_loss: 0.0060 - val_acc: 0.9627
Epoch 7/20
60000/60000 [==============================] - 4s 74us/step - loss: 0.0053 - acc: 0.9683 - val_loss: 0.0059 - val_acc

In [23]:
# 檢視準確率
score = model.evaluate(x_train, y_train, batch_size=10000)
print("Loss: %f" %score[0])
print("準確率: %f" %(score[1]*100))

60000/60000 [==============================] - 1s 22us/step
Loss: 0.003387
準確率: 98.071667
